### Table of contents

* **Classes**
  * Intro
  * Basics of classes
  * Static methods / Properties
  * Magic methods
  * Encapsulation
* **Intermediate Data Structures**

  * Dataclass
  * Enum
  * Named Tuple

# Classes (~20:02 (47min))

### Intro (~19:23 (8min))

Think of a *class* as a blueprint. It bundles **data** (what something *is*) together with **behavior** (what it *does*).


A “Chair” has properties like color/legs and actions like “can sit”. In code, those become **attributes** (data) and **methods** (actions).

In [ ]:
class Chair:
    def __init__(self, color: str, legs: int):
        self.color = color
        self.legs = legs

    def sit(self):
        print(f"Sitting on a {self.color} chair with {self.legs} legs")

chair = Chair("red", 4)
chair.sit()

##### The pain without classes

When data and behavior are separate, bugs creep in.

In [ ]:
# two accounts as plain dicts
alice = {"owner": "Alice", "balance": 100}
bob   = {"owner": "Bob",   "balance": 50}

def deposit(acct, amount):
    acct["balance"] += amount

def withdraw(acct, amount):
    if acct["balance"] >= amount:
        acct["balance"] -= amount

deposit(alice, 30)
withdraw(bob, 80)        # silently fails? returns nothing
bob["balnce"] = 9999     # typo creates a new key -> corrupted state


In [ ]:
# much better
class BankAccount:
    # __init__ runs automatically when you create an object
    def __init__(self, owner, balance=0):
        self.owner = owner     # attributes: per-object state
        self.balance = balance
        self.transactions = [] # list attribute; we can call its methods via self.x.append(...)

    # methods: verbs that act on the object
    def deposit(self, amount):
        self.balance += amount
        self.transactions.append(("deposit", amount))

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Not enough funds")
        self.balance -= amount
        self.transactions.append(("withdraw", amount))

alice = BankAccount("Alice", 100)
bob   = BankAccount("Bob", 50)

alice.deposit(30)
try:
    bob.withdraw(80)
except ValueError as e:
    print(e)  # Not enough funds


**attributes**

They hold each object’s own data. `alice.balance` and `bob.balance` are independent. Objects share the *methods’ code* but keep their *own state*

**methods**

They enforce your rules in one place (“can’t withdraw beyond balance”). If the rule changes, you update one method, not dozens of call sites—this is practical **encapsulation**

**self**

Inside a method, `self` is the object the method was called on. Calling
`alice.deposit(30)` is the same as `BankAccount.deposit(alice, 30)`. Keep `self` as the first parameter by convention.

### Basics of classes (~19:43 (20min))

In python, everything is an object.

object vs class / type

In [ ]:
# Instance
1


In [ ]:
# Class/type
(1).__class__, type(1)

In [ ]:
# Instance
int  # it's an object, instance of the class `type`


In [ ]:

# Class/type
(int).__class__, type(int), type(type)

**Classes: Syntax**

In [ ]:
class A:
    X = 10                       # Class variable

    def __init__(self, x: int):  # Initialization of the self instance state
        self._x = x              # Instance variable of self
    
    def foo(self) -> None:       # Class method
        self._x += 1             # Updating the self instance variable
        
    @staticmethod                # Static method of the class
    def bar() -> str:            # ! Do not pass the self instance
        return "bar"
    
    @classmethod                 # Class method (classmethod)
    def baz(cls: type) -> str:   # ! Instead of the self instance, the class cls is passed
        return cls.__name__
    
    @property                    # Property method of the class
    def x(self) -> int:            
        return self._x
    
    @x.setter                    # Property setter method of the class
    def x(self, x: int) -> None:
        self._x = x

##### Objects → Instantiation

A class is created / declared → An instance is constructed (`__new__`) → The instance is initialized (`__init__`)

In [ ]:
class A:
    """My class A"""
    def __init__(self): # != __new__ 
        print(f"Called init, {self}")
    
    def foo(self):
        print(f"Called foo, {self}")
        

In [ ]:
a = A(); b = A()

`__dict__` shows all attributes of the object

In [ ]:
A.__dict__

In [ ]:
a = A()
a.__dict__

##### Instance Attributes

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x
        
a = A(1); b = A(2)

In [ ]:
a.x, b.x

In [ ]:
# under the hood
a.__dict__, b.__dict__

Updating

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x
        
a = A(1); b = A(2)
a.x = 3
a.x, b.x


##### Methods

In [ ]:
class A:
    def __init__(self, x1: int):
        self.y, self.z = None, None
        self.x = x1
        
    def foo(self) -> None:
        self.x += 1

a = A(1); b = A(2)
b.foo()

In [ ]:
a.x, b.x

In [ ]:
# How to call foo directly from the class?
A.foo(a)  # equivalent to a.foo()

In [ ]:
a.x, b.x

optional

##### Class attributes

Class attributes allow to store data that is shared by all instances of the class. \
Most common use case is to store constants.

<div style="border: 1px solid green; background-color: #d4edda; padding: 10px; border-radius: 5px;">
  <div style="font-weight: bold"> Good practice</div> class attributes are named in uppercase and aren't changed after initialization.
</div>

In [ ]:
class A:
    GRAVITY: float = 9.81

a = A(); b = A()

In [ ]:
a.__dict__, b.__dict__

In [ ]:
A.__dict__

In [ ]:
a.GRAVITY  # accessible in self

In [ ]:
A.GRAVITY  # accessible from the class

Updating

In [ ]:
class A:
    GRAVITY: int = 9.81

a = A(); b = A()
A.GRAVITY = 3.86

In [ ]:
b.GRAVITY

In [ ]:
a.GRAVITY = 10
a.GRAVITY, b.GRAVITY, A.GRAVITY

In [ ]:
# Why so?
a.__dict__, b.__dict__

`a.GRAVITY = 10` created a new attribute for the instance `a`. \
After that, we have attribute with name `GRAVITY` in the instance `a` and in the class `A`. \
When we try to access `a.GRAVITY`, Python first checks if the attribute exists in the instance `a`. \
Only then Python checks if the attribute exists in the class `A`. \
`a.GRAVITY` is an instance attribute, `A.GRAVITY` is a class attribute.

optional

### Static methods / Properties (~19:51 (8min))

##### @staticmethod

In [ ]:
class A:
    @staticmethod
    def foo() -> int:
        return 1

a = A(); b = A()

In [ ]:
# How to access foo from the class and from the instance?
a.foo(), A.foo()

##### Use Case: @staticmethod → alternative __init__

In [ ]:
!touch /tmp/a.txt /tmp/b.txt /tmp/c.txt

In [ ]:
import os

class A:
    def __init__(self, folder: str, file_name: str):
        self.folder = folder
        self.file_name = file_name
    
    @staticmethod
    def from_path(path: str) -> 'A':
        folder, file_name = path.rsplit("/", 1)
        return A(folder, file_name)
    
    @staticmethod
    def from_folder(folder: str)-> list['A']:
        return [A(folder, filename) for filename in os.listdir(folder)]
        

a = A("/tmp", "a.txt")
b = A.from_path("/tmp/b.txt")
cs = A.from_folder("/tmp")

a.folder, b.folder, cs[0].__dict__

optional

### @property

In [ ]:
class A:
    def __init__(self, x: int):
        self._x = x

    @property
    def x(self) -> int:
        return self._x

a = A(1); b = A(2)

In [ ]:
a.x, b.x

In [ ]:
a.x = 3

In [ ]:
a.__dict__

In [ ]:
A.__dict__

##### @property → setter

In [ ]:
class A:
    def __init__(self, x: int):
        self._x = x
        
    @property
    def x(self) -> int:
        return self._x
    
    @x.setter
    def x(self, x: int) -> None:
        self._x = x

a = A(1); b = A(2)

In [ ]:
# a.x = 5
a.x = 10

In [ ]:
a.x

In [ ]:
a.__dict__

##### Use Case: a dynamic attribute

In [ ]:
class Student:
    def __init__(self, name: str, grades: dict | None = None):
        self.name = name
        self.grades = grades or {}

    def add_grade(self, subject: str, grade: int):
        self.grades[subject] = grade

    @property
    def gpa(self):
        return sum(self.grades.values()) / len(self.grades)

student = Student("John", {"Math": 5, "English": 4})
print(student.gpa)
student.add_grade("Physics", 3)
print(student.gpa)

##### Classes → summary

In [ ]:
class A:
    X: int = 10                  # Class variable

    def __init__(self, x: int):  # Initialization of the self instance state
        self._x = x              # Instance variable of self
    
    def foo(self) -> None:       # Class method
        self._x += 1             # Updating the self instance variable
        
    @staticmethod                # Static method of the class
    def bar() -> str:            # ! Do not pass the self instance
        return "bar"
    
    @classmethod                 # Class method (classmethod)
    def baz(cls) -> str:         # ! Instead of the self instance, the class cls is passed
        return cls.__name__
    
    @property                    # Property method of the class
    def x(self) -> int:            
        return self._x
    
    @x.setter                    # Property setter method of the class
    def x(self, x: int) -> None:
        self._x = x

In [ ]:
# Which of these are class attributes and which are object attributes?

### Encapsulation (~19:55 (4min))

**Encapsulation** is keeping an object’s data behind a small, safe interface—exposing only the methods you want—so you can enforce rules, prevent illegal states, and change internals without breaking other code.

**Why it matters (real world):** like an ATM: you press *withdraw*; you don’t edit the bank’s ledger—rules run inside.

**Anti-example**

```python
acct = {"balance": 100}
acct["balance"] = -500   # anyone can create an impossible state
```

**Encapsulated**

```python
class Account:
    def __init__(self, balance=0): self.__balance = balance  # hidden detail
    def balance(self): return self.__balance                  # read-only view
    def withdraw(self, amount):
        if amount > self.__balance: raise ValueError("insufficient")
        self.__balance -= amount
```


##### Classes → public/private labels

In [ ]:
class A:
    def __init__(self):
        self.x: int = 1     # Public
        self._x: int = 2    # Private
        self.__x: int = 3   # Super private

    def foo(self) -> str:   # Public
        return "foo"
    
    def _foo(self) -> str:  # Private
        return "_foo"
    
    def __foo(self) -> str: # Super private
        return "__foo"
    

a = A()

print(a.x)
print(a._x)
print(a._A__x)
print(a.foo())
print(a._foo())
print(a._A__foo())

Note: For people familiar with access modifiers in other languages, you can think of private attributes as protected, and super-private as private.

### Magic methods (~20:02 (7min))

Magic (dunder) methods of classes are methods that give them some properties. Their names start with two underscores (hence, dunder -- double underscore), for example -- method `__init__`. The call of magic methods in the interpreter happens implicitly.

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x

##### `__str__`/`__repr__`

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x


a = A(6)

print(a)
a

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x
        
    def __str__(self) -> str: # what for?
        return f"A with attr {self.x}"
    
    def __repr__(self) -> str:
        return f"A({self.x})"

a = A(6)

print(a)
a

In [ ]:
str(a), repr(a)

In [ ]:
class Polynomial:
    def __init__(self, coefficients: tuple):
        self.coefficients = coefficients 
    
    def __repr__(self) -> str:
        return "Polynomial" + str(self.coefficients)

    
    @staticmethod
    def x_expr(k: int) -> str:
        if k == 0:
            res = ""
        elif k == 1:
            res = "x"
        else:
            res = "x^"+str(k)
        return res

    def __str__(self) -> str:
        degree = len(self.coefficients) - 1
        res = ""
        for a_n, n in zip(self.coefficients, range(0, degree + 1)):
            if abs(a_n) == 1 and n < degree:
                sign = '+' if a_n > 0 else '-'
                res += f"{sign}{self.x_expr(degree - n)}"  
            elif a_n != 0:
                res += f"{a_n:+g}{self.x_expr(degree - n)}"

        return res.lstrip('+')

In [ ]:
polynom = Polynomial((1, 0, -4, 3, 1))

polynom

In [ ]:
print(polynom)

In [ ]:
from IPython.display import display, Math

display(Math(str(polynom)))

##### arithmetic

inplace: change the object in place \
non-inplace: create a new object

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x

    def __add__(self, other: 'A') -> 'A':  # non-inplace
        return A(self.x + other.x)
    
    def __iadd__(self, other: 'A') -> 'A':  # inplace
        self.x += other.x  # be aware of the semantics
        return self

a = A(6)
b = A(4)
id_A = id(a)

a += b
print(id(a) == id_A)

a = a + b
print(id(a) == id_A)

a.x

optional

##### `__call__`

In [ ]:
from math import factorial, sqrt

class Power:
    def __init__(self, p: float):
        self.p = p
        
    def __call__(self, a: float) -> float:
        return a**self.p
    
power = Power(3)
power(4)

##### `__len__`

In [ ]:
class PythonDudes:
    def __init__(self, names: list[str]):
        self.names = names
    
    def __len__(self) -> int:
        return len(self.names)
    
    def add(self, name: str) -> None:
        return self.names.append(name)
    
    
catalog = PythonDudes(["john", "jane", "jim"])
catalog.add("steven")
len(catalog), catalog.names

##### `__eq__`

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x
    
    def __eq__(self, other: 'A') -> bool:  
        return self.x == other.x
    
#     def __ne__(self, other: 'A') -> bool:  
#         return self.x != other.x

# a1 = A(3)
A(3) == A(3),  A(3) != A(5), A(3) != A(3),

# https://docs.python.org/3/reference/datamodel.html#object.__lt__
# https://stackoverflow.com/questions/4352244/should-ne-be-implemented-as-the-negation-of-eq

##### `__lt__`/`__gt__`

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x

    def __lt__(self, other: 'A') -> bool:
        print(self.x)
        return self.x < other.x

    def __gt__(self, other: 'A') -> bool:
        print(self.x)
        return self.x < other.x

        
A(5) < A(3),  A(5) > A(3)

##### `__le__`/`__ge__`

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x

    def __le__(self, other: 'A') -> bool:
        print(self.x)
        return self.x < other.x

        
A(5) <= A(3),  A(5) >= A(3)

# Intermediate Data Structures (~20:25 (23min))

### Dataclass (~20:13 (11min))

Objective: a data-holder class

In [ ]:
from dataclasses import dataclass

@dataclass
class A:
    x: int
    y: int

a = A(1, 2)
a

In [ ]:
@dataclass
class AWithMethods:
    x: int
    y: int
    
    def get_distance(self) -> int:
        return (self.x ** 2 + self.y ** 2) ** 0.5
        
a = AWithMethods(3, 4)
a.get_distance()

optional

Dataclasses are configurable: [more here](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass)

In [ ]:
# Let's play with the parameters, look into dicts

import typing as tp
from dataclasses import dataclass, field

@dataclass(init=True, repr=True, eq=True, order=True, unsafe_hash=True, frozen=True)
class Case:
    name: str = field(compare=True)
    given: int
    expected: object = field(default="test_name")

c = Case("test_name", 2, 1)

##### Use Case: A lot of arguments

In [ ]:
from dataclasses import dataclass
def function_with_a_lot_of_arguments(time, money, place, people):
    pass

In [ ]:
@dataclass
class ArgumentsContainer:
    time: int
    money: int
    place: str
    people: list[str]

def function_with_a_lot_of_arguments(args: ArgumentsContainer):
    pass

### Enum (~20:19 (6min))

Objective: a pool of constants

In [ ]:
from enum import Enum

class DeviceType(Enum):
    ANDROID = "android"
    WINDOWS = "windows"
    IOS = "ios"
    LINUX = "linux"
    OTHER = "other"

# https://www.python.org/dev/peps/pep-0435/#motivation

In [ ]:
type(DeviceType.ANDROID)

In [ ]:
d1 = DeviceType.ANDROID
d2 = DeviceType.ANDROID
d1 == d2, d1 is d2

In [ ]:
DeviceType.ANDROID.name, DeviceType.ANDROID.value

In [ ]:
{DeviceType.ANDROID: 1, DeviceType.WINDOWS: 2}

##### Auto

In [ ]:
from enum import auto

class DeviceType(Enum):
    ANDROID = auto()
    WINDOWS = auto()
    IOS = auto()
    LINUX = auto()
    OTHER = auto()

In [ ]:
DeviceType.ANDROID, DeviceType.WINDOWS, DeviceType.IOS

### Named Tuple (~20:25 (6min))

In [ ]:
from collections import namedtuple

Point = namedtuple("Point", ["x", "y"])

p = Point(1, 2)

print(p.x)
print(p.y)

In [ ]:
p.x = 3
print(p.x)

You can't change the attributes of a namedtuple (immutable). \
You can't have methods in a namedtuple. \
You can't specify the type of the attributes.

Almost always, dataclasses are better than namedtuples.